In [5]:
!pip install --upgrade pip

In [4]:
# Cell 1: ติดตั้งไลบรารี
import sys
!{sys.executable} -m pip install selenium webdriver_manager
print("ติดตั้งไลบรารีเสร็จสิ้น (หากติดตั้งไปแล้ว จะแสดงว่า Requirement already satisfied)")

  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl.metadata (1.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------- ----------------------- 3.9/9.4 MB 23.8 MB/s eta 0:00:01
   -------------------------------------- - 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import sys
print(sys.executable)

c:\Users\ball-pc\AppData\Local\Programs\Python\Python313\python.exe


In [4]:
# --- Cell 1: Imports และการตั้งค่า ---
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from dotenv import load_dotenv

# โหลดค่าจากไฟล์ .env
load_dotenv()

# --- ค่าคงที่ ---
LOGIN_URL = "https://cenproject.rid.go.th/"
YOUR_USERNAME = os.getenv("CEN_USERNAME")
YOUR_PASSWORD = os.getenv("CEN_PASSWORD")
EXCEL_FILE_PATH = 'data.xlsx' # <<<<---- ระบุชื่อไฟล์ Excel ของคุณที่นี่
OUTPUT_EXCEL_PATH = 'result_data.xlsx' # <<<<---- ไฟล์ผลลัพธ์ที่จะถูกสร้างขึ้น

print("การตั้งค่าเสร็จสิ้น")

การตั้งค่าเสร็จสิ้น


In [5]:
print(YOUR_USERNAME, YOUR_PASSWORD)

rid2ez2 wa792d


In [6]:
# --- Cell 2: เริ่มต้น WebDriver และ Login ---
print("กำลังเริ่มต้น WebDriver...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()

print(f"กำลังเข้าสู่หน้า Login: {LOGIN_URL}")
driver.get(LOGIN_URL)

wait = WebDriverWait(driver, 5) # เพิ่มเวลารอเป็น 20 วินาทีเผื่อเว็บโหลดช้า

try:
    # 1. Login
    print("กำลังรอหน้า Login...")
    username_field = wait.until(EC.presence_of_element_located((By.NAME, "Username")))
    password_field = wait.until(EC.presence_of_element_located((By.NAME, "Password")))
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))

    print("กำลังกรอกข้อมูล Login...")
    username_field.send_keys(YOUR_USERNAME)
    password_field.send_keys(YOUR_PASSWORD)
    login_button.click()
    print("Login สำเร็จ")

    # 2. กดเข้าระบบติดตาม
    print("กำลังรอลิงก์ 'ระบบติดตาม'...")
    system_tracking_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[./img[@alt='banner-track']]")))
    system_tracking_link.click()
    print("คลิก 'ระบบติดตาม' สำเร็จ")

    # --- ส่วนที่แก้ไข ---
    # 3. รอให้หน้ากาก Loading หายไปก่อน
    print("กำลังรอให้หน้า Dashboard โหลดเสร็จ (รอให้ Loading Overlay หายไป)...")
    loading_overlay_xpath = "//div[contains(@style, 'display: block') and contains(@style, 'z-index: 9999')]"
    # เราจะรอจนกว่า Element นี้จะ "มองไม่เห็น" (invisible)
    wait.until(EC.invisibility_of_element_located((By.XPATH, loading_overlay_xpath)))
    print("Loading Overlay หายไปแล้ว หน้าเว็บพร้อมใช้งาน")

    # 4. ตอนนี้ค่อยคลิกเมนูงบประมาณ
    print("กำลังรอเมนู 'งบประมาณ'...")
    budget_menu_left_nav = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'งบประมาณ')]")))
    budget_menu_left_nav.click()
    print("คลิก 'งบประมาณ' สำเร็จ")
    
    # 5. รอให้ตารางข้อมูลหลักโหลดเสร็จ (ยังคงสำคัญ)
    # wait.until(EC.presence_of_element_located((By.ID, "DataTables_Table_0"))) 
    print("หน้าตางบประมาณโหลดเรียบร้อยแล้ว")

except Exception as e:
    print(f"เกิดข้อผิดพลาดระหว่างการ Login หรือการนำทางเริ่มต้น: {e}")
    # driver.quit() # อาจจะคอมเมนต์ไว้ก่อนตอนทดสอบ

กำลังเริ่มต้น WebDriver...
กำลังเข้าสู่หน้า Login: https://cenproject.rid.go.th/
กำลังรอหน้า Login...
กำลังกรอกข้อมูล Login...
Login สำเร็จ
กำลังรอลิงก์ 'ระบบติดตาม'...
คลิก 'ระบบติดตาม' สำเร็จ
กำลังรอให้หน้า Dashboard โหลดเสร็จ (รอให้ Loading Overlay หายไป)...
Loading Overlay หายไปแล้ว หน้าเว็บพร้อมใช้งาน
กำลังรอเมนู 'งบประมาณ'...
คลิก 'งบประมาณ' สำเร็จ
หน้าตางบประมาณโหลดเรียบร้อยแล้ว


In [ ]:
# --- Cell 4: วนลูปค้นหาโครงการและคลิก Action ---
# สมมติว่า driver และ wait ถูกสร้างและพร้อมใช้งานจาก cell ก่อนหน้า
# และ EXCEL_FILE_PATH, OUTPUT_EXCEL_PATH ถูกกำหนดค่าไว้แล้ว

print("--- เริ่มต้นกระบวนการใน Cell 4 ---")

# อ่านข้อมูลจาก Excel
try:
    df = pd.read_excel(EXCEL_FILE_PATH)
    if 'Status' not in df.columns:
        df['Status'] = ''
    if 'ErrorMessage' not in df.columns:
        df['ErrorMessage'] = ''
    print(f"อ่านไฟล์ '{EXCEL_FILE_PATH}' สำเร็จ พบ {len(df)} รายการ")
except FileNotFoundError:
    print(f"!!! ไม่พบไฟล์ '{EXCEL_FILE_PATH}' กรุณาตรวจสอบและรันใหม่ !!!")
    # หยุดการทำงานถ้าไม่พบไฟล์
    raise # ส่ง error ต่อไปเพื่อหยุดการทำงานของ Notebook

# วนลูปทีละแถวใน DataFrame
for index, row in df.iterrows():
    # ข้ามรายการที่ทำสำเร็จใน step ก่อนๆ แล้ว (ถ้ามี)
    # หรือจะเริ่มใหม่ทั้งหมดก็ได้โดยลบคอมเมนต์นี้ออก
    # if 'Clicked' in str(row['Status']): 
    #     print(f"ข้ามรายการ {index + 1} '{row['ProjectName']}' เพราะเคยคลิกแล้ว")
    #     continue

    print(f"\n===== [รายการที่ {index + 1}/{len(df)}] กำลังค้นหา: {row['ProjectName']} =====")
    
    try:
        project_name = row['ProjectName']

        # Step 4.1: ค้นหาแถว (tr) ของโครงการที่ต้องการ
        # XPath นี้จะหาแถวที่มีคอลัมน์ (td) ที่ 2 ที่มีข้อความตรงกับชื่อโครงการ
        # td[2] มาจากโครงสร้าง <td>1</td><td>ชื่องาน...</td>
        project_row_xpath = f"//tbody/tr[td[2][contains(., '{project_name}')]]"
        
        print(f"  > กำลังรอให้แถวของ '{project_name}' ปรากฏ...")
        # เราใช้ visibility_of_element_located เพื่อให้แน่ใจว่าแถวนั้นมองเห็นได้และไม่ได้ถูกซ่อนอยู่
        project_row = wait.until(EC.visibility_of_element_located((By.XPATH, project_row_xpath)))
        print("  > พบแถวโครงการแล้ว")

        # Step 4.2: ค้นหาปุ่ม "เรียกดูข้อมูลแถบเดิม" และคลิก
        # เราจะใช้ data-tip ที่เป็นเอกลักษณ์และเสถียรที่สุด
        action_button_xpath = ".//button[@data-tip='เรียกดูข้อมูลแถบเดิม']"
        
        action_button = project_row.find_element(By.XPATH, action_button_xpath)
        print("  > พบปุ่ม Action แล้ว, กำลังคลิก...")
        
        action_button.click()
        
        print(f"  > คลิก Action ของ '{project_name}' สำเร็จแล้ว")

        # --- จุดพักสำหรับขั้นตอนต่อไป ---
        # หลังจากคลิกแล้ว รายการย่อยจะปรากฏขึ้น
        # เราต้องใส่โค้ดสำหรับ Step 5 (ค้นหารายการย่อย) ต่อที่นี่
        
        # ใส่การหน่วงเวลาเล็กน้อยเพื่อให้เว็บมีเวลาแสดงผลรายการย่อย
        time.sleep(1.5) # เพิ่มเวลาเล็กน้อยเผื่อ animation ซับซ้อน
        
        # อัปเดตสถานะใน DataFrame ว่าคลิกสำเร็จ
        df.loc[index, 'Status'] = 'Action 1 Clicked'
        df.loc[index, 'ErrorMessage'] = ''

    except TimeoutException:
        error_msg = f"ไม่พบโครงการ '{project_name}' บนหน้าเว็บ (หมดเวลารอ)"
        print(f"  > !!! ERROR: {error_msg}")
        df.loc[index, 'Status'] = 'Failed: Project Not Found'
        df.loc[index, 'ErrorMessage'] = error_msg
        continue # ข้ามไปทำรายการถัดไป

    except NoSuchElementException:
        error_msg = f"พบแถวโครงการ แต่ไม่พบปุ่ม Action 'เรียกดูข้อมูลแถบเดิม'"
        print(f"  > !!! ERROR: {error_msg}")
        df.loc[index, 'Status'] = 'Failed: Action Button Not Found'
        df.loc[index, 'ErrorMessage'] = error_msg
        continue # ข้ามไปทำรายการถัดไป

    except Exception as e:
        error_msg = str(e).splitlines()[0]
        print(f"  > !!! ERROR: เกิดข้อผิดพลาดที่ไม่คาดคิด: {error_msg}")
        df.loc[index, 'Status'] = 'Failed: Unknown Error'
        df.loc[index, 'ErrorMessage'] = error_msg
        # หากเกิด error ที่ไม่คาดคิด อาจจะต้องโหลดหน้าใหม่เพื่อ reset สถานะ
        # driver.get(TARGET_BUDGET_URL)
        # wait.until(EC.presence_of_element_located((By.TAG_NAME, "tbody")))
        continue

    finally:
        # บันทึกผลลัพธ์ลง Excel ทุกๆ 10 รายการ หรือเมื่อทำรายการสุดท้ายเสร็จ
        if (index + 1) % 10 == 0 or (index + 1) == len(df):
            print(f"===== กำลังบันทึกความคืบหน้าลงในไฟล์ '{OUTPUT_EXCEL_PATH}' =====")
            df.to_excel(OUTPUT_EXCEL_PATH, index=False, engine='openpyxl')

print("\n===== กระบวนการใน Cell 4 เสร็จสิ้น =====")

--- เริ่มต้นกระบวนการใน Cell 4 ---
!!! ไม่พบไฟล์ 'data.xlsx' กรุณาตรวจสอบและรันใหม่ !!!


Exception ignored in: <function Service.__del__ at 0x0000020277DA1620>
Traceback (most recent call last):
  File "c:\Users\const\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 196, in __del__
    self.stop()
  File "c:\Users\const\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 152, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\const\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 137, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\const\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\const\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdri

FileNotFoundError: [Errno 2] No such file or directory: 'data.xlsx'